Explainable AI
What we will cover today:
XAI or Explainable AI
Integration in Machine Learning
Intuition about Shapley Values
Shapley values using the SHAP package
Local explainability
Global explainability
Explaining image classification
1️⃣ XAI or Explainable AI
Why do we need XAI?
Try explaining to non-data-scientists...
... a linear regression 🤔

^
y
=
β
0
+
β
1
x
1
+
β
2
x
2
+
⋯
... a decision tree 😬

... a Random Forest model 😵‍💫

... a deep learning model 🤯

Where do we need XAI?
Can I get a loan?

What drives predictions?


Examples of application areas
Model validation
Model debugging
Knowledge discovery
Legal requirements:
🇪🇺 GDPR limitations on automated decision making based on personal data without human intervention
Interpretability vs explainability
Interpretability
Understand exactly why and how the model is generating predictions
You need to observe the inner mechanics of the model
Explainability
Explain the behavior of the model in human terms
With complex models you cannot fully understand what lead to the prediction
But you can use techniques to discover meaning between input data and model outputs
🔗 Interpretability versus explainability - AWS
2️⃣ Integration in ML
Where does XAI fit in the ML solution?


Approaches
Intrinsic explainable models
The model parameters directly explain how the features contribute to the prediction
Typical examples: linear regression, decision tree, KNN
Agnostic approach:
The ML model is a black box: XAI does not assume any knowledge of the model used
It only relies on inputs and predictions of the model
Typical use: random forest, RNN
Model-dependent approach
Knowledge of the ML model is used to produce explanations
Typical examples: an agnostic approach being optimized (for speed) for certain ML models
Global vs local explanation

Source: Model Explainability — How to choose the right tool? — Mateusz Garbacz
3️⃣ Shapley values — intuition
Foundations in game theory
Game theory is the study of mathematical models of strategic interactions among rational agents
Game theory	Non-cooperative games	Cooperative games

John von Neumann
(1903-1957)	John Nash
(1928-2015)	Lloyd Shapley
(1923-2016)
On the Theory of Games of Strategy (1928)	Non-cooperative games (1950)	Notes on the n-Person
Game -- II: The Value
of an n-Person Game (1950)
Nobel Prize in Economic Sciences (1994)	Nobel Prize in Economic Sciences (2012)
Cooperative games

From game theory to machine learning


* Actually the difference between the prediction and the average prediction
Intuition

Shapley values
Imagine a dataset with
n
features
x
0
,
x
1
,
…
,
x
j
,
…
,
x
n
Now, take a particular instance (i.e. one row in your data)
Each feature
x
j
contributes to the prediction of this particular instance
The Shapley value
ϕ
j
(
i
n
s
t
a
n
c
e
)
=
the contribution of feature value j for this instance
For a specific instance, there are as many Shapley values as there are features
Shapley values are specific for an instance
A different instance, will have different Shapley values
Prediction for x
= Average prediction
+ sum of Shapley values for each contribution
How are Shapley values calculated?
Let's calculate the Shapley value for one feature
j
for one particular instance

Let's build all the coalitions of features, and look at feature
j
's contribution to the prediction
The simplest coalition, is the empty one ...

We compare the coalition with and without feature
j
We calculate both predictions and take the difference
... followed by a coalition of only one feature ...

Again, we compare the coalition with and without feature
j
Again, we calculate both predictions and take the difference
... and so on

Each time we compare the coalition with and without feature
j
Each time we calculate both predictions and take the difference
... and finally we calculate average of these differences = the Shapley value
🤯  Wait a minute...
We can't just drop a feature from our model, right❓
🪄 We need to simulate the exclusion of a feature
💡 To do so we make multiple predictions
👉 Each time we replace the excluded feature by a randomly drawn value 🎲
🤓 Want to know more? 🔗
4️⃣ Shapley in practice with the SHAP package
SHAP

SHAP or Shapley Additive exPlanations is a Python package
Basically, it generates a local explaining model for a chosen instance
It explains a single prediction through a linear combination (👉 additive)
of the underlying Shapley values
SHAP also offers global explainability extensions
🔗 Documentation
Let's put it into practice with 🏠🏠🏠 and XGBoost 🚀
# Our classic imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import datasets, ensemble
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import xgboost
# !pip install shap
import shap
# Run the following command. Needed for some visualizations.
shap.initjs();

Fit a model
We are using California house prices from 1990, in $100.000
# Load the data
X, y = shap.datasets.california()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=13)

# Fit our model
model_tree = xgboost.XGBRegressor(n_estimators=100, max_depth=2)
model_tree.fit(X_train, y_train)

# Predict
y_test_pred = model_tree.predict(X_test)
Let's have a look at the predictions
# The average house price (in $100.000)
print(f"The average house price in the train set is {y_train.mean():.3f}")


# Have a look at the mean squared error on the test set
mse = mean_squared_error(y_test, y_test_pred)
print(f"The Mean Squared Error on the test set is   {mse:.3f}")
The average house price in the train set is 2.067
The Mean Squared Error on the test set is   0.292
# Let's check our predictions on the test set visually
plt.scatter(y_test, y_test_pred)
plt.title("Prediction vs true value")
plt.xlabel("y_test"); plt.ylabel("y_test_predict");

5️⃣ Local explainability
Select an instance, and calculate the prediction
row_to_show = 24  # Select one instance to explain
feature_values = X_test.iloc[[row_to_show]]
prediction = model_tree.predict(feature_values)
print(f"{'The prediction for this instance:':<35}{prediction[0]:>7.3f}")
The prediction for this instance:    1.558
And calculate the SHAP values
# Step 1: Create a SHAP Explainer
explainer = shap.Explainer(model_tree)
# Step 2: Calculate the SHAP values
# Using the explainer we just created
# and giving as input the feature values for our instance
shap_values_one = explainer(feature_values)
# Have a look at these values
print(f"{'Base value:':<35}{shap_values_one.base_values[0]:>7.3f}")
print(f"{'Sum of SHAP values:':<35}{shap_values_one.values.sum():>7.3f}")
print(f"{'The prediction for this instance:':<35}{prediction[0]:>7.3f}")
Base value:                          2.067
Sum of SHAP values:                 -0.509
The prediction for this instance:    1.558
Now that we have our SHAP values, let's visualize them
shap.plots.bar(shap_values_one[0])

shap.plots.waterfall(shap_values_one[0])

shap.plots.force(shap_values_one[0])
0.6674
0.8674
1.067
1.267
1.467
1.667
1.867
2.067
2.267
2.467
2.667
2.867
3.067
3.267
3.467
AveBedrms = 1.104
Latitude = 33.95
MedInc = 2.203
AveOccup = 4.035
AveRooms = 4.413
Longitude = -118.3
base value
1.56
1.56
higher
→
f(x)
←
lower
6️⃣ Global explainability
# Let's calculate SHAP values for our whole test population
# We can use the same explainer
# This time we feed it our whole test population

shap_values = explainer(X_test)
shap.plots.bar(shap_values)

Or as a beeswarm plot
shap.plots.beeswarm(shap_values)

The SHAP package has even more visualisations
👉 Have a look at the 🔗 docs
7️⃣ What about images?
SHAP also works for images 🤩
Preparing a model
Let's use a pretrained ResNet50 model
# !pip install opencv-python
import requests
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input, decode_predictions
# Loading the class names from ImageNet 1000
url = "https://s3.amazonaws.com/deep-learning-models/image-models/imagenet_class_index.json"
class_names = requests.get(url).json().values()
class_names = [value[1] for value in class_names]
# Load the pre-trained model and some sample data
model_deep = ResNet50(weights='imagenet')
X, y = shap.datasets.imagenet50()
X = X.astype(int)
Obtaining the SHAP values
# Function to preprocess the data and get the model output
# We will use this as input for the SHAP explainer
def model(X):
    X_copy = X.copy()
    X_copy = preprocess_input(X_copy)
    return model_deep(X_copy)

# A masker that will mask out partitions of the input image
masker = shap.maskers.Image("blur(128,128)", X[0].shape)

# Finally create the explainer
explainer = shap.Explainer(model, masker, output_names=class_names)

# Explain some images using 500 evaluations of the model
# to estimate the SHAP values
shap_values = explainer(X[1:5], max_evals=500, batch_size=50,
                        outputs=shap.Explanation.argsort.flip[:4],
                        silent=True)
shap.image_plot(shap_values, pixel_values=X[1:5])

Further reading 📚:
Explainable AI with Python — Leonida Gianfagna, Antonio Di Cecco

Interpretable Machine Learning — A Guide for Making Black Box Models Explainable — Christoph Molnar

SHAP package documentation
